In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

### Test Local LangGraph Agent

In [ ]:
from langgraph_sdk import get_client

url = "https://leetmock-ts-fa225f46565756e7b0567441810f232f.default.us.langgraph.app"
client = get_client(url=url)

# Using the graph deployed with the name "agent"
assistant_id = "test-agent"

# create thread
thread = await client.threads.create()

print(thread)

In [3]:
openai_assistant = await client.assistants.create(graph_id=assistant_id)

In [ ]:
openai_assistant

In [ ]:
input = {"messages": []}

async for chunk in client.runs.stream(
    thread["thread_id"],
    assistant_id,
    input={ "foo": "brian!" },
    stream_mode="values",
    # multitask_strategy="interrupt",
):
    # tags = chunk.data.get("tags", [])
    # if "chatbot" not in tags:
    #     continue

    # event = chunk.data.get("event", None)
    # if event != "on_chat_model_stream":
    #     continue

    # content = chunk.data.get("data", {}).get("chunk", {}).get("content", "")

    print("-" * 100)
    print(chunk)
    print("-" * 100)

In [1]:
from agent_graph.template.graph import create_graph

graph = create_graph()

In [7]:
async for chunk in graph.astream(
    input={
        "messages": []
    },
    stream_mode=["values", "updates"],
    config={ "configurable": {"thread_id": "1" } }
):
    print(chunk)

('values', {'messages': [HumanMessage(content='(User just joined the call, please welcome and introduce yourself:)', id='bc34fd39-a0b5-49ae-9287-d11fb7863ad9'), AIMessage(content="Hello and welcome! We're thrilled to have you here. I'm [Your Name], and I'm here to assist you with any questions or topics you'd like to discuss. Feel free to share a bit about yourself or let me know how I can help you today!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 19, 'total_tokens': 70, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_482c22a7bc', 'finish_reason': 'stop', 'logprobs': None}, id='run-d35ce49d-8891-463e-bd1d-ea3e45f0763a-0', usage_metadata={'input_tokens': 19, 'output_tokens': 51, 'total_tokens': 70})], 'initialized': True, 'trigger': False})
